In [1]:
import boto3
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
import snowflake.connector
import os
from snowflake.connector.pandas_tools import write_pandas

In [2]:
load_dotenv()

True

In [3]:
s3_bucket_name = 's3-snowflake-etl-pipeline'
s3_file_key = 'data/AIRBNB_NYC.csv'

In [4]:
# AWS Credentials
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

In [5]:
# Initialize S3 Client and Extract CSV from S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [6]:
def load_data_from_s3(bucket, key):
    response = s3_client.get_object(Bucket=bucket, Key=key)
    data = response['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(data))
    return df

In [7]:
df = load_data_from_s3(s3_bucket_name, s3_file_key)
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [8]:
# Transform the Data with Pandas (Modify as Needed)
df['top_price'] = df['price'] * 2

In [9]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,top_price
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,298
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,450
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,300
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,178
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,160


In [10]:
# Snowflake connection parameters
sf_user = os.getenv('SNOWFLAKE_USER')
sf_password = os.getenv('SNOWFLAKE_PASSWORD')
sf_account = os.getenv('SNOWFLAKE_ACCOUNT')
sf_warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
sf_database = os.getenv('SNOWFLAKE_DATABASE')
sf_schema = os.getenv('SNOWFLAKE_SCHEMA')
sf_table = os.getenv('SNOWFLAKE_TABLE')

# Create a connection to Snowflake
cnx = snowflake.connector.connect(
    user=sf_user,
    password=sf_password,
    account=sf_account,
    warehouse=sf_warehouse,
    database=sf_database,
    schema=sf_schema,
)

In [11]:
# Load Data into Snowflake using write_pandas
try:
    success, nchunks, nrows, _ = write_pandas(cnx, df, sf_table, auto_create_table=True, overwrite=True)
    
    if success:
        print(f"Data successfully loaded into Snowflake table '{sf_table}' with {nrows} rows.")
    else:
        print("Failed to load data into Snowflake.")
except Exception as e:
    print("Error loading data into Snowflake:", e)
finally:
    cnx.close()

Data successfully loaded into Snowflake table 'airbnb_nyc' with 48895 rows.
